In [212]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import math
from sklearn import preprocessing
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold

In [213]:
heart_data= pd.read_csv("heart.csv")

In [215]:
print(str(len(heart_data.index)))

270


In [216]:
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,2
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,1
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,2
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,1
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,1


In [217]:
heart_data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [218]:
pd.get_dummies(heart_data['target'])

,1,2
0,0,1
1,1,0
2,0,1
3,1,0
4,1,0
...,...,...
265,1,0
266,1,0
267,1,0
268,1,0


In [221]:
print(heart_data['target'])

0      2
1      1
2      2
3      1
4      1
      ..
265    1
266    1
267    1
268    1
269    2
Name: target, Length: 270, dtype: int64


In [222]:
# Transform into binary classification
heart_data['target'] = [1 if b==2 else 0 for b in heart_data.target]
heart_data['target'].value_counts()

0    150
1    120
Name: target, dtype: int64

In [223]:
df_majority = heart_data[heart_data.target==0]
df_minority = heart_data[heart_data.target==1]

# Downsample majority class
df_majority_downsampled = resample(df_majority,
                                 replace=False,    # sample without replacement
                                 n_samples=120,     # to match minority class
                                 random_state=123) # reproducible results
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
# Display new class counts
df_downsampled.target.value_counts()
# 1    49
# 0    49
# Name: balance, dtype: int64

1    120
0    120
Name: target, dtype: int64

In [314]:
# Separate input features (X) and target variable (y)
# y = df_downsampled.target
# X = df_downsampled.drop('target', axis=1)
array = df_downsampled.values

X = array[:,0:13]
y = array[:,13]


In [225]:
print(heart_data)

     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     70    1   4       130   322    0        2      109      0      2.4   
1     67    0   3       115   564    0        2      160      0      1.6   
2     57    1   2       124   261    0        0      141      0      0.3   
3     64    1   4       128   263    0        0      105      1      0.2   
4     74    0   2       120   269    0        2      121      1      0.2   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
265   52    1   3       172   199    1        0      162      0      0.5   
266   44    1   2       120   263    0        0      173      0      0.0   
267   56    0   2       140   294    0        2      153      0      1.3   
268   57    1   4       140   192    0        0      148      0      0.4   
269   67    1   4       160   286    0        2      108      1      1.5   

     slope  ca  thal  target  
0        2   3     3       1  
1        2   0     7     

In [226]:
from sklearn.model_selection import train_test_split

In [239]:
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
X_train_minmax = min_max_scaler.fit_transform(heart_data)

yNormalized = min_max_scaler.fit_transform(X)
print(xNormalized)
yNormalized = min_max_scaler.fit_transform(y.reshape(-1, 1))


[[0.16544537 0.0030638  0.00919141 ... 0.0030638  0.         0.02144662]
 [0.13592579 0.00261396 0.00261396 ... 0.00522792 0.         0.0182977 ]
 [0.14065176 0.00299259 0.00897777 ... 0.00299259 0.         0.00897777]
 ...
 [0.16467889 0.00283929 0.00567858 ... 0.00567858 0.         0.00851787]
 [0.15562367 0.00324216 0.00648432 ... 0.00972648 0.         0.02269512]
 [0.1905785  0.00284446 0.01137782 ... 0.00568891 0.00853337 0.00853337]]


In [277]:
#feature selection
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0)

# rfe = RFE(clf, 6)
# rfe = rfe.fit(xNormalized, yNormalized)
# clf.fit(xNormalized, yNormalized)
# print(clf.feature_importances_)
# print(rfe.support_)
# test = SelectKBest(score_func=chi2, k=6)
# fit = test.fit(X, y)
# features = fit.transform(X)
# Summarize selected features
# print(features[0:5,:])
# print(rfe.transform(xNormalized,2))

In [315]:
X_train, X_test, y_train, y_test = train_test_split(xNormalized, yNormalized, test_size=0.4, random_state=1)

In [316]:
clf.fit(X_train, y_train.ravel())


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [317]:
predictions = clf.predict(X_test)

In [318]:
from sklearn.metrics import classification_report
classification_report(y_test,predictions)

'              precision    recall  f1-score   support\n\n         0.0       0.88      0.83      0.85        53\n         1.0       0.80      0.86      0.83        43\n\n    accuracy                           0.84        96\n   macro avg       0.84      0.85      0.84        96\nweighted avg       0.85      0.84      0.84        96\n'

In [319]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,predictions)

array([[44,  9],
       [ 6, 37]])

In [320]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predictions)*100

84.375

In [321]:
# Train model
clf_2 = LogisticRegression().fit(X_train, y_train)
 
# Predict on training set
pred_y_2 = clf_2.predict(X_test)
 
# Is our model still predicting just one class?
print( np.unique( pred_y_2 ) )
# [0 1]
 
# How's our accuracy?
print( accuracy_score(y_test, pred_y_2) )
# 0.581632653061

[0. 1.]
0.625


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
